In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pathdata17zip = '/content/drive/My Drive/Code_Attribution/data17.zip'

In [0]:
import zipfile
zip_ref = zipfile.ZipFile(pathdata17zip, 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
datasetPath = '/tmp/data17'

In [5]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau, TensorBoard
from keras.models import Model
import keras.backend as K

Using TensorFlow backend.


In [0]:
import os

path = datasetPath

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.cpp' in file:
            files.append(os.path.join(r, file))

# for f in files:
#     print(f)

In [7]:
len(files)

56837

In [0]:
lengths = []
minLength = 2000

In [0]:
def padding(fname):
    f = open(fname, "r")
    y = f.read()
    f.close()
    
    origy = y
    while (len(y) < minLength):
        y += origy
        
    f = open(fname, "w")
    f.write(y)
    f.close()
    return y

In [0]:
def rem_dir(path):
  for r, d, f in os.walk(path):
    for fil in f:
      if fil in files:
        files.remove(os.path.join(r, fil))

In [0]:
outputvar = {}
counter = [0]

In [0]:
def makeInOut(path, _len, counter):
  count = 0
  for r, d, f in os.walk(path):
    name = r.split('/')[-1]
    for fil in f:
      _fil = os.path.join(r, fil)
      if name not in outputvar.keys():
        outputvar[name] = counter[0]
        counter[0] += 1
      if _fil in files:
        if count < (0.8*_len):
          if count < (0.2*_len):
            x_val.append(_fil)
            y_val.append(outputvar[name])
          else:
            x_train.append(_fil)
            y_train.append(outputvar[name])
          count += 1
        else:
          x_test.append(_fil)
          y_test.append(outputvar[name])

In [0]:
x_train = []
x_test = []
y_train = []
y_test = []
x_val = []
y_val = []
split = 0.8

In [14]:
for i in range(0,3):
    print(i)
    lengths = []
    count = 0
    for fil in files:
        f = open(fil, "r")
        x = f.read()
        y = x.split()
        f.close()
        if(len(x) <= 100):
            files.remove(fil)
        elif len(x) == 0:
            files.remove(fil)
        elif(len(y) < minLength):
            x = padding(fil)
            lengths.append(len(x))
        else:
            lengths.append(len(x))


0
1
2


In [0]:
def check(path, counter):
  for r, d, f in os.walk(path):
    _len = len(f)
    if _len < 5:
      rem_dir(path)
    else:
      makeInOut(path, _len, counter)

In [0]:
for r, d, f in os.walk(datasetPath):
  for di in d:
    check(os.path.join(r, di), counter)

In [17]:
_check = []
for x in y_train:
  if x not in _check:
    _check.append(x)
print(len(_check))
print(counter)
print(len(outputvar))

5065
[5065]
5065


In [18]:
len(files)

56808

In [19]:
print(max(lengths))
print(min(lengths))
print(len(lengths))
print(len(files))

95319
2000
56808
56808


In [20]:
print(x_train[:20])
print(x_test[:10])
print(y_train[:30])
print(y_test[:10])
print(x_val[:10])
print(y_val[:10])

['/tmp/data17/JohnSmith/b0.cpp', '/tmp/data17/JohnSmith/a0.cpp', '/tmp/data17/JohnSmith/a1.cpp', '/tmp/data17/JohnSmith/c1.cpp', '/tmp/data17/JohnSmith/b10.cpp', '/tmp/data17/JohnSmith/b1.cpp', '/tmp/data17/simonlindholm/stalls2.cpp', '/tmp/data17/simonlindholm/fashion0.cpp', '/tmp/data17/simonlindholm/fashion1.cpp', '/tmp/data17/simonlindholm/shoot1.cpp', '/tmp/data17/simonlindholm/beam1.cpp', '/tmp/data17/simonlindholm/mountain1.cpp', '/tmp/data17/simonlindholm/cruise1.cpp', '/tmp/data17/simonlindholm/stable0.cpp', '/tmp/data17/simonlindholm/fresh1.cpp', '/tmp/data17/simonlindholm/pony1.cpp', '/tmp/data17/simonlindholm/stalls1.cpp', '/tmp/data17/simonlindholm/stacks0.cpp', '/tmp/data17/simonlindholm/pancakes1.cpp', '/tmp/data17/simonlindholm/dice1.cpp']
['/tmp/data17/JohnSmith/c2.cpp', '/tmp/data17/simonlindholm/dice0.cpp', '/tmp/data17/simonlindholm/tidy1.cpp', '/tmp/data17/simonlindholm/stalls0.cpp', '/tmp/data17/simonlindholm/shoot0.cpp', '/tmp/data17/simonlindholm/news1.cpp', '/t

In [0]:
import numpy as np

In [0]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_val = np.array(x_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [24]:
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

indices = np.arange(x_test.shape[0])
np.random.shuffle(indices)
x_test = x_test[indices]
y_test = y_test[indices]

indices = np.arange(x_val.shape[0])
np.random.shuffle(indices)
x_val = x_val[indices]
y_val = y_val[indices]

print(x_train[:20])
print(x_test[:10])
print(y_train[:30])
print(y_test[:10])
print(x_val[:10])
print(y_val[:10])

['/tmp/data17/goryudyuma/main (2)0.cpp' '/tmp/data17/shopnobaj/A0.cpp'
 '/tmp/data17/mingcl/roller1.cpp' '/tmp/data17/vova/main_middle1.cpp'
 '/tmp/data17/yashkumar18/solution11.cpp' '/tmp/data17/Stun/a1.cpp'
 '/tmp/data17/natufunu/C1.cpp' '/tmp/data17/ayushj1201/Cj32.cpp'
 '/tmp/data17/harababurel/brut0.cpp' '/tmp/data17/Galieve/A_c1.cpp'
 '/tmp/data17/maopao/A0.cpp'
 '/tmp/data17/kawaiigabrielneko/rollercoaster0.cpp'
 '/tmp/data17/jeremy624/Untitled30.cpp' '/tmp/data17/sonlh07/C_op0.cpp'
 '/tmp/data17/krodhking/D0.cpp' '/tmp/data17/Lightmoon/B1.cpp'
 '/tmp/data17/Frost./CC0.cpp' '/tmp/data17/shubh.400076/tidy1.cpp'
 '/tmp/data17/failedcoder/20171bA0.cpp' '/tmp/data17/kakalan/pA0.cpp']
['/tmp/data17/Jaesung/1BB1.cpp' '/tmp/data17/threat/C20.cpp'
 '/tmp/data17/Miceren/C0.cpp'
 '/tmp/data17/quynhtruong1304/OversizedPancakeFlipper0.cpp'
 '/tmp/data17/bshankar/C0.cpp' '/tmp/data17/nemkin/af1.cpp'
 '/tmp/data17/sgyang/b1.cpp' '/tmp/data17/infinity1/B_small0.cpp'
 '/tmp/data17/Xenoframium/m

In [0]:
def walk(path, vocab, count):
    for r, d, f in os.walk(path):
        for file in f:
            f = open(os.path.join(r, file))
            x = f.read()
            for line in x:
                for letter in line:
                    if letter not in vocab.keys():
                        vocab[letter] = count[0]
                        count[0] += 1


In [26]:
vocab = {}
count = [0]
path = datasetPath
for r, d, f, in (os.walk(path)):
    for i, dir in enumerate(d):
        walk(os.path.join(r, dir), vocab, count)
print(count)

[1468]


In [27]:
i = 0
for word in vocab.keys():
  print(word)
  i += 1
  if i >= 10:
    break

/


 
g
o
l
e
c
d
j


In [0]:
def getFiles(path):
    matrix = []
    for p in path:
        tmp = np.zeros((50, 40), dtype = int)
        f = open(p)
        for i in range(0,50):
            for j in range(0, 40):
                tmp[i][j] = vocab[f.read(1)]
        matrix.append(tmp)
    return np.array(matrix)

In [0]:
num_classes = len(outputvar)
input_shape = (50, 40)
batch_size = 1024
def data_generator(file_paths, author_numbers):
    batch_start = 0
    batch_end = batch_start + batch_size
    n = file_paths.shape[0]
    indexes = np.arange(0, n, batch_size)
    if n % batch_size != 0:
        indexes = indexes[:-1] 
    while True:
        np.random.shuffle(indexes)
        for ind in indexes:
            batch_start = ind
            batch_end = batch_start + batch_size
            myFiles = file_paths[batch_start:batch_end]
            batch_files = getFiles(myFiles)
            batch_authors = author_numbers[batch_start:batch_end]
            batch_x = np.zeros((batch_size, 50, 40))
            tmp = np.zeros((batch_size, num_classes))
            batch_x = np.array( batch_files )
            for i, j in enumerate(batch_authors):
                tmp[i][j] = 1
            batch_y = np.array( tmp )
            yield( batch_x, batch_y )

In [30]:
input_fil = Input(shape=(input_shape), name='input_fil')
conv = Conv1D(64, kernel_size=7, strides = 2,  activation='relu')(input_fil)
maxP = MaxPooling1D()(conv)
flat = Flatten()(maxP)
output_class = Dense(num_classes, activation='softmax')(flat)

model = Model(inputs=input_fil, outputs=output_class)

W0726 10:57:45.534170 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 10:57:45.575175 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 10:57:45.576699 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 10:57:45.615293 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [31]:
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=adam, 
              metrics=['accuracy'])

model.summary()

W0726 10:57:45.673683 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 10:57:45.682445 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_fil (InputLayer)       (None, 50, 40)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 64)            17984     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 704)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5065)              3570825   
Total params: 3,588,809
Trainable params: 3,588,809
Non-trainable params: 0
_________________________________________________________________


In [0]:
train_gen = data_generator(x_train, y_train)
test_gen = data_generator(x_test, y_test)
val_gen = data_generator(x_val, y_val)

In [33]:
hist1 = model.fit_generator(train_gen, epochs=10, steps_per_epoch=len(y_train)//batch_size, 
                           validation_data=val_gen, validation_steps=len(y_val)//batch_size, verbose=1)

W0726 10:57:45.826838 140631057315712 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0726 10:57:45.897816 140631057315712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
23/23 [==============================] - 42s 2s/step - loss: 13.4789 - acc: 7.2181e-04 - val_loss: 8.5707 - val_acc: 9.7656e-04
Epoch 2/10
23/23 [==============================] - 35s 2s/step - loss: 8.5436 - acc: 0.0022 - val_loss: 8.5562 - val_acc: 0.0020
Epoch 3/10
23/23 [==============================] - 36s 2s/step - loss: 8.3935 - acc: 0.0087 - val_loss: 8.3922 - val_acc: 0.0068
Epoch 4/10
23/23 [==============================] - 36s 2s/step - loss: 7.4437 - acc: 0.0390 - val_loss: 7.9652 - val_acc: 0.0459
Epoch 5/10
23/23 [==============================] - 36s 2s/step - loss: 5.2969 - acc: 0.2184 - val_loss: 7.2202 - val_acc: 0.1851
Epoch 6/10
23/23 [==============================] - 36s 2s/step - loss: 2.6038 - acc: 0.5529 - val_loss: 6.6371 - val_acc: 0.3661
Epoch 7/10
23/23 [==============================] - 36s 2s/step - loss: 1.0260 - acc: 0.8049 - val_loss: 6.6630 - val_acc: 0.4405
Epoch 8/10
23/23 [==============================] - 35s 2s/step - loss: 0.4758 - 

In [35]:
xtestdata = getFiles(x_test)
print(xtestdata.shape)

(6240, 50, 40)


In [36]:
preds = model.predict(xtestdata)
print(preds.shape)

(6240, 5065)


In [37]:
myPreds = []
for i in preds:
    myPreds.append(np.argmax(i))
myPreds = np.array(myPreds)
print(myPreds.shape)

(6240,)


In [38]:
count = 0
for i in range(myPreds.shape[0]):
    if myPreds[i] == y_test[i]:
        count += 1

acc = (count / myPreds.shape[0]) * 100
print("Accuracy", acc)

Accuracy 49.118589743589745
